In [35]:
import cv2 #opencv读取的格式是BGR
import numpy as np
import matplotlib.pyplot as plt#Matplotlib是RGB
%matplotlib inline 

In [36]:
def cv_show(name,img):
    cv2.imshow(name,img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
    

In [37]:
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

In [38]:
def order_points(rect,box):
    rect_angle = np.zeros((4, 2), dtype = "float32")
    left_top = box[2]
    right_top = box[3]
    left_bottom = box[1]
    right_bottom = box[0]
    theta = rect[2]
    height =  np.int0(rect[1][1])
    width =  np.int0(rect[1][0])
    return [left_top,right_top,right_bottom,left_bottom,height,width]

In [39]:
def four_point_transform(image, rect,pts):
    # 获取输入坐标点
    rect_ = order_points(rect,pts)
    (tl, tr, br, bl,height,width) = rect_
    points = np.array([(tl, tr, br, bl)],dtype = "float32")
    # 变换后对应坐标位置
    dst =np.array([
        [0, 0],
        [height, 0],
        [height, width],
        [0,width]], dtype = "float32")
         
    # 计算变换矩阵
    print(points)
    M = cv2.getPerspectiveTransform(points, dst)
    warped = cv2.warpPerspective(image, M, (height, width))

    # 返回变换后结果
    return warped

In [71]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 5))
#sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

In [81]:
img_o = cv2.imread('ISBN -7-80603-718-7.jpg',0) 
img_o = resize(img_o, height = 500)
img = cv2.imread('ISBN -7-80603-718-7.jpg',0) #0表示灰度图
ratio = img.shape[0] / 500.0
img = resize(img, height = 500)
img = cv2.GaussianBlur(img, (5, 5), 1)
cv_show('img',img)
img = img[:int(img.shape[0]*0.8),:]
cv_show('img',img)

In [84]:
#gradX = cv2.Sobel(img, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=-1)
#gradX = np.absolute(gradX)
#(minVal, maxVal) = (np.min(gradX), np.max(gradX))
#gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
ret, img_binary = cv2.threshold(img, 0 , 255, cv2.THRESH_OTSU)
#gradX = gradX.astype("uint8")
#cv_show('gradX',gradX)
img_binary = cv2.bitwise_not(img_binary)
cv_show('a',img_binary)
img_binary = cv2.dilate(img_binary,rectKernel,iterations = 2)
#ret, img_binary = cv2.threshold(img_binary, 0 , 255, cv2.THRESH_OTSU)
cv_show('img',img_binary)

In [74]:
edged = cv2.Canny(img_binary, 100, 200)
cnts,a = cv2.findContours(edged, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [75]:
char_region =[]
for (i, c) in enumerate(cnts):
    # 计算矩形
    rect  = cv2.minAreaRect(c)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, (255, 255, 255), 5)
    #cv2.imwrite('contours.png', img.copy())
    #print(rect)
    #print(rect[2],box)
    (w,h) = rect[1][0],rect[1][1]
    ar = h/w
    if ar >= 8: 
        cv2.drawContours(img, [box], 0, (255, 255, 255), 5)
        cv2.imwrite('contours.png', img.copy())
        char_region = [rect,box]
        cv_show('a',img.copy())
        print((w,h))
        break

(44.561927795410156, 489.6343078613281)


In [76]:
type(char_region[0])

tuple

In [77]:
char_region

[((343.7901611328125, 46.96757888793945),
  (44.561927795410156, 489.6343078613281),
  -84.53463745117188),
 array([[585,  92],
        [ 97,  45],
        [102,   1],
        [589,  48]], dtype=int64)]

In [78]:
img_o

array([[179, 180, 181, ...,   0,   0,   0],
       [181, 182, 178, ...,   0,   0,   0],
       [181, 177, 165, ...,   0,   0,   1],
       ...,
       [192, 198, 198, ...,  50,  42,  30],
       [192, 190, 188, ...,  49,  39,  24],
       [197, 195, 196, ...,  50,  37,  23]], dtype=uint8)

In [79]:
warped = four_point_transform(img_o, char_region[0],char_region[1])
#warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
print("STEP 3: 变换")
cv_show('pcture',warped)

[[[102.   1.]
  [589.  48.]
  [585.  92.]
  [ 97.  45.]]]
STEP 3: 变换
